Notebook to make the results Latex table 

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [3]:
all_bursts_bary = pd.read_csv("../data/all_bursts_bary.csv")
all_bursts_bary = all_bursts_bary.drop("Unnamed: 0", axis=1)

<IPython.core.display.Javascript object>

In [4]:
def get_number(x, dx):
    """
    Returns a string of the measurement value
    together with the measurement error
    x: measurement value
    dx: measurment error
    """

    # Power of dx
    power_err = np.log10(dx)

    # Digits of dx in format a.bcd
    n_err = dx / (10 ** np.floor(power_err))

    # If the second digit in dx is >=5
    # round the 1st digit in dx up
    if n_err % 1 >= 0.5:
        # If the first digit of dx is 9
        # the precision is one digit less
        if int(n_err) == 9:
            err = 1
            # The precision of x is determined by the precision of dx
            prec = int(-np.floor(np.log10(dx))) - 1
        else:
            err = np.ceil(n_err)
            # The precision of x is determined by the precision of dx
            prec = int(-np.floor(np.log10(dx)))
    # Otherwise round down
    else:
        err = np.floor(n_err)
        # The precision of x is determined by the precision of dx
        prec = int(-np.floor(np.log10(dx)))

    return x, err, prec

<IPython.core.display.Javascript object>

In [5]:
def get_err_string(x, le, ue):
    """
    Estimates proper rounding
    and returns both errors with
    necessary significant digits

    """
    min_err = min(le, ue)
    x, err, prec = get_number(x, min_err)
    s = ""
    if min_err > 1:
        s += str(int(x))
        les = int(np.round(le, prec))
        ues = int(np.round(ue, prec))
    else:
        s += str(np.round(x, prec))
        les = np.round(le, prec)
        ues = np.round(ue, prec)
    ret = f"${s}^" + "{+" + str(ues) + "}_{-" + str(les) + "}$"
    return ret

<IPython.core.display.Javascript object>

In [6]:
def get_printable_values(row, col_names, append_string="", i=0):
    s = row["bidx"]

    # highlight bursts with unconstrained fluences
    if not row["use_fluence"] and row["fit_method"] == "mcmc":
        s += "$\dagger$"

    # highlight bursts that used curvefit
    if row["fit_method"] == "curvefit":
        s += "$*$"

    # print!
    for col in col_names:
        s += " & "
        try:
            val = row[col + append_string]
        except KeyError:
            s += "-"
            continue

        if np.isnan(val):
            s += "-"
        else:
            ue = row["upper error" + col + append_string]
            le = row["lower error" + col + append_string]
            if np.isnan(le):
                le = ue
            if "$\mu_t$ (ms)" in col:
                e = (ue + le) / 2
                x, err, prec = get_number(val, e)
                s += "{:.{prec}f}({:.0f})".format(x, err, prec=prec)
            else:
                s += get_err_string(val, le, ue)
    s += "\\\\"
    return s, row

<IPython.core.display.Javascript object>

In [7]:
col_names = all_bursts_bary.columns[:-5][::3]

<IPython.core.display.Javascript object>

In [8]:
col_names

Index(['$\mu_f$ (MHz)', '$\sigma_f$ (MHz)', '$S$ (Jy ms)', '$\mu_t$ (ms)',
       '$\sigma_t$ (ms)', '$\tau$ (ms)', 'DM (pc cm$^{-3}$)'],
      dtype='object')

<IPython.core.display.Javascript object>

Print table header 

In [9]:
s = ""
s += "\\" + "tablehead{"
s += "\\" + "colhead{Burst}"
for col in col_names:
    s += " & \\" + "colhead{" + col.split(" (")[0] + "}"
s += "\\\\"
s += "\n"
s += "\\" + "colhead{Idx.}"
for col in col_names:
    s += " & \\" + "colhead{(" + col.split(" (")[1] + "}"
s += "}"
print(s)

\tablehead{\colhead{Burst} & \colhead{$\mu_f$} & \colhead{$\sigma_f$} & \colhead{$S$} & \colhead{$\mu_t$} & \colhead{$\sigma_t$} & \colhead{$\tau$} & \colhead{DM}\\
\colhead{Idx.} & \colhead{(MHz)} & \colhead{(MHz)} & \colhead{(Jy ms)} & \colhead{(ms)} & \colhead{(ms)} & \colhead{(ms)} & \colhead{(pc cm$^{-3}$)}}


<IPython.core.display.Javascript object>

Print Table

In [10]:
for i, row in all_bursts_bary.iterrows():
    s, row = get_printable_values(row, col_names, append_string="")
    print(s)

B1$*$ & $1558^{+30}_{-30}$ & $212^{+40}_{-40}$ & $0.09^{+0.02}_{-0.02}$ & 57644.408906976(1) & $0.0^{+0.02}_{-0.02}$ & $1.9^{+0.3}_{-0.3}$ & $565.3^{+0.4}_{-0.4}$\\
B2$*$ & $1201^{+10}_{-10}$ & $50^{+10}_{-10}$ & $0.043^{+0.007}_{-0.007}$ & 57644.40956768(1) & $1.35^{+0.05}_{-0.05}$ & - & $562.4^{+0.8}_{-0.8}$\\
B3.1$\dagger$ & $2944^{+300}_{-600}$ & $751^{+300}_{-200}$ & $0.6^{+0.7}_{-0.3}$ & 57644.409673699(3) & $0.4^{+0.2}_{-0.2}$ & $1.3^{+0.7}_{-0.7}$ & $566.8^{+0.8}_{-0.9}$\\
B3.2$\dagger$ & $1146^{+300}_{-1400}$ & $1268^{+2000}_{-1000}$ & $0.09^{+0.13}_{-0.04}$ & 57644.40967384(2) & $0.3^{+1.4}_{-0.2}$ & $0.3^{+0.7}_{-0.1}$ & $564.7^{+1.5}_{-0.4}$\\
B4$\dagger$ & $3074^{+200}_{-600}$ & $549^{+80}_{-110}$ & $2^{+4}_{-2}$ & 57644.410072889(4) & $1.1^{+0.2}_{-0.2}$ & $0.3^{+0.2}_{-0.1}$ & $563^{+1}_{-1}$\\
B5$\dagger$ & $2100^{+900}_{-1600}$ & $2703^{+900}_{-1200}$ & $0.19^{+0.05}_{-0.04}$ & 57644.410157834(4) & $0.7^{+0.3}_{-0.3}$ & $1.0^{+0.4}_{-0.3}$ & $562.1^{+0.7}_{-0.6}$\\
B6.

<IPython.core.display.Javascript object>

In [11]:
all_bursts_bary.columns

Index(['$\mu_f$ (MHz)', 'upper error$\mu_f$ (MHz)', 'lower error$\mu_f$ (MHz)',
       '$\sigma_f$ (MHz)', 'upper error$\sigma_f$ (MHz)',
       'lower error$\sigma_f$ (MHz)', '$S$ (Jy ms)', 'upper error$S$ (Jy ms)',
       'lower error$S$ (Jy ms)', '$\mu_t$ (ms)', 'upper error$\mu_t$ (ms)',
       'lower error$\mu_t$ (ms)', '$\sigma_t$ (ms)',
       'upper error$\sigma_t$ (ms)', 'lower error$\sigma_t$ (ms)',
       '$\tau$ (ms)', 'upper error$\tau$ (ms)', 'lower error$\tau$ (ms)',
       'DM (pc cm$^{-3}$)', 'upper errorDM (pc cm$^{-3}$)',
       'lower errorDM (pc cm$^{-3}$)', 'cand_id', 'comp_num', 'fit_method',
       'use_fluence', 'bidx'],
      dtype='object')

<IPython.core.display.Javascript object>

In [12]:
1 / 0

ZeroDivisionError: division by zero

<IPython.core.display.Javascript object>